In [2]:
pip install opencv-python


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.3/38.8 MB ? eta -:--:--
   - -------------------------------------- 1.0/38.8 MB 3.6 MB/s eta 0:00:11
   -- ------------------------------------- 2.4/38.8 MB 4.8 MB/s eta 0:00:08
   --- ------------------------------------ 3.7/38.8 MB 5.2 MB/s eta 0:00:07
   ---- ----------------------------------- 4.7/38.8 MB 5.2 MB/s eta 0:00:07
   ----- ---------------------------------- 5.5/38.8 MB 5.0 MB/s eta 0:00:07
   -------- ------------------------------- 7.9/38.8 MB 6.2 MB/s eta 0:00:06
   -------- ------------------------------- 8.4/38.8 MB 5.8 MB/s eta 0:00:06
   --------- ------------------------------ 9.4/38.8 MB 5.9 MB/s eta 0:00:05
   ---------- ----------------------------- 10.5/38.8 MB 5.5 MB/s eta 0:00:06
   ------------ --------------------------- 11.8/38.8 MB 5.6 MB/s eta 0:00:05
   -------

In [22]:
import cv2
import numpy as np
import xml.etree.ElementTree as ET


In [23]:
image_path = 'test_image.png'  # Replace with the actual path to the image

# Load the image in grayscale (or IMREAD_COLOR for color)
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
integral_image = cv2.integral(image)


In [24]:
cv2.imshow('Test Image',image)

In [26]:
# Function to compute the sum of a rectangular region using the integral image
def compute_sum(integral_img, top_left, bottom_right):
    x1, y1 = top_left
    x2, y2 = bottom_right
    
    # Compute the sum of pixels in the rectangle using the integral image
    return integral_img[y2, x2] - integral_img[y1, x2] - integral_img[y2, x1] + integral_img[y1, x1]


In [27]:
# Step 3: Define a function to compute the Haar-like feature for a window
def detect_haar_feature(integral_img, window_top_left, feature_size):
    x, y = window_top_left
    w, h = feature_size
    
    # Define feature rectangles relative to the top-left corner of the window
    # For a two-rectangle horizontal Haar feature
    top_sum = compute_sum(integral_img, (x, y), (x + w, y + h // 2))
    bottom_sum = compute_sum(integral_img, (x, y + h // 2), (x + w, y + h))
    
    # Haar feature value (difference between top and bottom sums)
    haar_feature_value = top_sum - bottom_sum
    
    return haar_feature_value

In [28]:
def sliding_window_haar_detection(integral_img, window_size, feature_size, threshold):
    img_height, img_width = integral_img.shape
    feature_locations = []  # List to store detected feature locations
    
    # Create the root element for the XML structure
    root = ET.Element("HaarFeatureDetections")
    
    # Slide window across the image
    for y in range(0, img_height - window_size[1], 2):  # Step size of 2 for faster scanning
        for x in range(0, img_width - window_size[0], 2):
            # Compute the Haar feature for the current window
            haar_value = detect_haar_feature(integral_img, (x, y), feature_size)
            
            # Check if feature value exceeds the threshold
            if haar_value > threshold:
                # Store the detection in the XML format
                feature = ET.SubElement(root, "Feature")
                
                # Store the window location (top-left corner)
                ET.SubElement(feature, "Location").text = f"({x}, {y})"
                
                # Store the feature value
                ET.SubElement(feature, "Value").text = str(haar_value)
                
                # Store the window size
                ET.SubElement(feature, "WindowSize").text = f"{window_size[0]}x{window_size[1]}"
                
                # Add the location to the feature_locations list
                feature_locations.append((x, y))  # Store the (x, y) coordinates
    
    # Return the XML tree structure and feature locations
    return root, feature_locations

In [29]:
# Pretty-print the XML for readability
def indent_xml(elem, level=0):
    i = "\n" + level * "  "
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "  "
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for subelem in elem:
            indent_xml(subelem, level + 1)
        if not subelem.tail or not subelem.tail.strip():
            subelem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [31]:
feature_size = (16, 16)
window_size = (16,16)
threshold = 50  # Example threshold for feature detection

# Apply the sliding window to find Haar feature locations
haar_feature_xml, detected_locations = sliding_window_haar_detection(integral_image, window_size, feature_size, threshold)

#Indent the XML for better readability
indent_xml(haar_feature_xml)

# Convert the XML structure to a string and save to a file
tree = ET.ElementTree(haar_feature_xml)
with open("haar_feature_detections.xml", "wb") as xml_file:
    tree.write(xml_file)
    
print("Detected Feature Locations:", detected_locations)
print("XML file saved with Haar feature locations.")

Detected Feature Locations: [(70, 0), (72, 0), (74, 0), (76, 0), (78, 0), (80, 0), (82, 0), (84, 0), (70, 2), (72, 2), (74, 2), (76, 2), (78, 2), (80, 2), (82, 2), (84, 2), (70, 4), (72, 4), (74, 4), (76, 4), (78, 4), (80, 4), (82, 4), (84, 4), (34, 6), (36, 6), (38, 6), (40, 6), (42, 6), (44, 6), (46, 6), (48, 6), (50, 6), (52, 6), (54, 6), (56, 6), (66, 6), (68, 6), (70, 6), (72, 6), (74, 6), (76, 6), (78, 6), (80, 6), (82, 6), (34, 8), (36, 8), (38, 8), (40, 8), (42, 8), (44, 8), (46, 8), (48, 8), (50, 8), (52, 8), (54, 8), (56, 8), (58, 8), (60, 8), (62, 8), (64, 8), (66, 8), (68, 8), (70, 8), (72, 8), (74, 8), (76, 8), (78, 8), (80, 8), (22, 10), (24, 10), (26, 10), (28, 10), (30, 10), (32, 10), (34, 10), (36, 10), (38, 10), (40, 10), (42, 10), (44, 10), (46, 10), (48, 10), (50, 10), (52, 10), (54, 10), (56, 10), (58, 10), (60, 10), (62, 10), (64, 10), (66, 10), (68, 10), (70, 10), (72, 10), (74, 10), (76, 10), (78, 10), (12, 12), (14, 12), (16, 12), (18, 12), (20, 12), (22, 12), 